In [63]:
import pandas as pd
import numpy as np  
import sys

import PyUber


In [64]:
xeus_source = 'F32_PROD_XEUS'


In [130]:
query = '''
    SELECT DISTINCT
        LOT7
    FROM
        F_LOT
    WHERE
        PRODUCT LIKE '8PXMCV%G'
        AND LOT_TITLE LIKE 'NPI% LL%'
'''

In [131]:
with PyUber.connect(datasource=xeus_source) as conn:
    df = pd.read_sql(query, conn)
# df.to_csv(r'debug\test_query.csv', index=False)

unique_lot7 = df['LOT7'].unique()
print(f"Unique LOT7 values: {unique_lot7}")

lot7_like_str = ', '.join([f"'{lot7}'" for lot7 in unique_lot7])
print(f"LOT7 string for SQL LIKE: {lot7_like_str}")

C:\Users\dagarcia\AppData\Local\Temp\ipykernel_28892\1420038716.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Unique LOT7 values: ['A528013']
LOT7 string for SQL LIKE: 'A528013'


In [132]:
query = f'''
    SELECT DISTINCT
        lf.OPERATION
        ,lf.OPER_SHORT_DESC AS OPER_SHORT
        ,o.oper_long_desc AS OPER_LONG
        ,o.area AS AREA
        ,o.module AS MODULE
        ,MIN(lf.EXEC_SEQ) AS SEQ
    FROM
        F_LOT_FLOW lf
        CROSS JOIN F_Facility f
        INNER JOIN F_Operation O ON o.operation=lf.operation AND o.facility = f.facility AND o.latest_version = 'Y'
    WHERE
        LOT7 IN ({lot7_like_str})
        AND LENGTH(LOT) = 8
    GROUP BY
        lf.OPERATION
        ,lf.OPER_SHORT_DESC
        ,o.oper_long_desc
        ,o.area
        ,o.module

'''

In [133]:

with PyUber.connect(datasource=xeus_source) as conn:
    df = pd.read_sql(query, conn)
    
df['OPERATION'] = df['OPERATION'].astype(int)
df['SEQ'] = df['SEQ'].astype(int)
df.to_csv(r'debug\test_query.csv', index=False)

C:\Users\dagarcia\AppData\Local\Temp\ipykernel_28892\874466524.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [134]:
df2 = df.sort_values(by='SEQ').reset_index(drop=True)


In [135]:
df2.dtypes

OPERATION      int64
OPER_SHORT    object
OPER_LONG     object
AREA          object
MODULE        object
SEQ            int64
dtype: object

In [136]:

# Define module lists
modules_18A = [
    "LI-SAVli", "LI-SAYli", "LI-SBHcu", "LI-SBLcu", "LI-SNEli", "LI-SNYli"
]

modules_10nm = [
    "LI-BE-193", "LI-BE-SED", "LI-BE-WET", "LI-FE-193", "LI-PD-WET",
    "LI-SSAFI-WET", "LI-WET", "LI-FE-248"
]


In [137]:

first_row = df2.iloc[0]
first_row


OPERATION                     261369
OPER_SHORT                T 8BSKRSTG
OPER_LONG     T P1278 BESKAR STAGING
AREA                      THIN FILMS
MODULE                  TF-TADro-STG
SEQ                                1
Name: 0, dtype: object

In [138]:

df3 = df2[
    (df2['SEQ'] == df2['SEQ'].min()
     | df2['OPERATION'] == 9812
     | ((df2['AREA'] == 'LITHO')
    #    & df['MODULE'].isin(modules_18A + modules_10nm)
       )
    )
]

# df2.to_csv(r'debug\test_query.csv', index=False)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [141]:
df3 = df2[(df2['OPERATION'] == 9812)
          | (df2['SEQ'] == df2['SEQ'].min())
          | ((df2['AREA']=='LITHO') & (df['MODULE'].isin(modules_18A + modules_10nm))
             
          )
         ]

In [156]:
df3 = df2.copy() 
df3['PC_STARTS_flag'] = False
df3['OPERATION_flag'] = False
# df3['AREA_flag'] = False
df3['MODULE_flag'] = False
df3.loc[df3['MODULE'] == "PC-STARTS", 'PC_STARTS_flag'] = True
df3.loc[df3['OPERATION'] == 9812, 'OPERATION_flag'] = True
# df3.loc[df3['AREA'] == 'LITHO', 'AREA_flag'] = True
df3.loc[df3['MODULE'].isin(modules_18A + modules_10nm), 'MODULE_flag'] = True

df3 = df3[df3[['PC_STARTS_flag', 'OPERATION_flag', 'MODULE_flag']].any(axis=1)].reset_index(drop=True)


df3['ORDER'] = range(len(df3))
df3 = df3.drop(columns=['PC_STARTS_flag', 'OPERATION_flag', 'MODULE_flag','SEQ','MODULE','AREA'])

In [169]:
def get_layer(row):
    
        
    if row['OPER_LONG'].find('START') != -1: return 'START'
    if row['OPER_LONG'].find('PACK') != -1: return 'SHIP'
    
    value = row['OPER_SHORT']

    cond_list = [' ','#',
                'L58','L5B','L52','L46','L4H','L4','L5', #10nm conditions
                'L8xr','L8c','L8s','L8b','L86','L81','L8d','L8' #18A conditions
                ] 
    for cond in cond_list:
        value = value.replace(cond,'')
    
    value = value[:3]

    if row['OPER_LONG'].find(value) == -1:
        if value[0]=='M':
            value = 'MT' + value[1]
        else:
            value = 'VA' + value[1]    
    
    return value

df3['LAYER'] = df3.apply(lambda row: get_layer(row), axis=1 )